### IMPORTAR BIBLIOTECAS

In [34]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### FONTE DE DADOS

Os dados utilizados para esse projeto foram retirados do kaglle, através do link: https://www.kaggle.com/marcusrb/uber-peru-dataset

O histórico de cotação (Real x PEN), foi retirado do link: https://br.investing.com/currencies/pen-brl-historical-data


### EXPLICAÇÃO SOBRE OS DADOS

1) UBER_PERU - Esse conjunto de dados apresenta 23.111 corridas através do aplicativo de mobilidade Uber, no país Peru. Cada linha apresenta informações sobre o serviço prestado.

2) PEN_BRL - Histórico de cotação da moeda PEN em Real.

Os valores do data set estão na moeda PEN (Novo Sol Peruano) no ano em que os serviços foram prestados (2010). Para uma melhor interpretação, a moeda será convertida para Real no valor da epoca.

Em seguida farei a importação dos dados, alguns tratamentos, visualizações e extração de insights.

### ENCONTRANDO (EM REAL) O VALOR DA MOEDA 'PEN' NO ANO DE 2010

O DataSet apresenta 5 colunas e representam a variação da moeda ao longo do ano de 2010.

Adotaremos a coluna "Último" (fechamento do preço no mês) como base de cálculo para a média do ano.

In [35]:
pen_brl = pd.read_csv('PEN_BRL_2010.csv')
pen_brl.head(3)

,Data,Último,Abertura,Máxima,Mínima,Var%
0,Dez 10,"0,5910","0,6050","0,6105","0,5905","-2,38%"
1,Nov 10,"0,6054","0,6074","0,6204","0,5975","-0,36%"
2,Out 10,"0,6076","0,6051","0,6162","0,5895","0,41%"


Isolei a coluna que preciso, em seguia analiso linhas em branco e o tipo de dado.

Foi observado que os dados da coluna estão como tipo 'Object', portanto será necessário a conversão.

In [36]:
pen_brl = pd.DataFrame(pen_brl['Último'])
display(pen_brl.head(3))
display(pen_brl.info())


,Último
0,"0,5910"
1,"0,6054"
2,"0,6076"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Último  12 non-null     object
dtypes: object(1)
memory usage: 224.0+ bytes


None

Fiz a conversao dos valores utilizando as funções apply e lambda, que percorre todo o data set, substituindo virgula por ponto.
O valor ainda foi convertido em float.

In [37]:
pen_brl['Último'] = pen_brl['Último'].apply(lambda x: float(x.replace(',', '.')))
display(pen_brl.head(3))
display(pen_brl.info())


,Último
0,0.5910
1,0.6054
2,0.6076


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Último  12 non-null     float64
dtypes: float64(1)
memory usage: 224.0 bytes


None

Por fim, armazenei a média dos preços em uma variável. Como conversão da moeda, adotaremos a média do ano.

In [38]:
brl_2010 = round(pen_brl['Último'].mean(), 2)
display(brl_2010)


0.62

Agora podemos inciar a análise dos dados ded corrida do aplicativo.

### TRABALHANDO NOS DADOS DO APLICATIVO UBER

Nossos dados estão em CSV. Iremos armazenar em uma variável (df). O df apresenta 23.111 registros e 28 colunas, das quais explicarei em seguida.

In [47]:
df = pd.read_csv('uber_peru_2010.csv', sep = ';')
display(df.head(3))
display(df.shape)


,journey_id,user_id,driver_id,taxi_id,icon,start_type,start_at,start_lat,start_lon,end_at,...,price_distance,price_duration,distance,duration,cost,cost_distance,cost_duration,source,driver_score,rider_score
0,23a1406fc6a11d866e3c82f22eed4d4c,0e9af5bbf1edfe591b54ecdfd7e91e26,583949a89a9ee17d19e3ca4f137b6b4c,b12f4f09c783e29fe0d0ea624530db56,executive,asap,16/11/2010 16:44,"-12,13983536","-77,02355957",16/11/2010 17:29,...,3626.0,195.0,11331.0,234.0,0.0,0.0,0.0,iPhone,5.0,5.0
1,dd2af4715d0dc16eded53afc0e243577,a553c46e3a22fb9c326aeb3d72b3334e,NaN,NaN,executive,asap,01/06/2010 0:34,"-12,13874817","-76,99536133",01/06/2010 0:37,...,NaN,NaN,0.0,0.0,0.0,NaN,NaN,iPhone,NaN,NaN
2,dd91e131888064bf7df3ce08f3d4b4ad,a553c46e3a22fb9c326aeb3d72b3334e,NaN,NaN,executive,asap,31/05/2010 5:01,"-12,12453079","-77,02780151",31/05/2010 5:04,...,NaN,NaN,0.0,0.0,0.0,NaN,NaN,iPhone,NaN,NaN


(23111, 28)

Vou usar .info() para analisar a qualidade do DF no que diz respeito a valores nulos, faltantes e o formato das variáveis.

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23111 entries, 0 to 23110
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   journey_id        23111 non-null  object 
 1   user_id           23111 non-null  object 
 2   driver_id         19726 non-null  object 
 3   taxi_id           19726 non-null  object 
 4   icon              23111 non-null  object 
 5   start_type        23111 non-null  object 
 6   start_at          23111 non-null  object 
 7   start_lat         23111 non-null  object 
 8   start_lon         23111 non-null  object 
 9   end_at            22835 non-null  object 
 10  end_lat           23111 non-null  object 
 11  end_lon           23111 non-null  object 
 12  end_state         23099 non-null  object 
 13  driver_start_lat  19621 non-null  object 
 14  driver_start_lon  19621 non-null  object 
 15  arrived_at        17716 non-null  object 
 16  currency          23111 non-null  object

Em seguida explano o significado das variáveis do data frame.

Algumas descrições podem não estar corretas, não encontrei boas referências explicando as variáveis.


COLUNA|DESCRIÇÃO|OBSERVAÇÃO|DECISÃO
---|---|---|---
journey_id|Id único referente a cada viagem||Não useremos
user_id|Id único referente a passageiro do app||Não useremos
driver_id|Id único referente a cada motorista do app||Não useremos
taxi_id|Id único referente ao veículo||Não useremos
icon|Categoria da viagem|*|
start_type|Como a viagem foi iniciada|*|
start_at|Partida - Data e hora||Irei separar data da hora
start_lat|Partida - Latitude||
start_lon|Partida - Longitude||
end_at|Chegada - Data e hora||Irei separar data da hora
ent_lat|Chegada - Latitude||
end_lon|Chegada - Longitude||
end_state|Status da viagem|*|
driver_start_lat|Motorista - Latitude inicial||
driver_start_lon|Motorista - Longitude incial||
arrived_at|Motorista - Horário de chegada na origem||Não useremos
currency|Moeda|PEN - Novo Sol Peruano|
price|Preço total da viagem||
price_distance|Preço parcial cobrado pela distância percorrida||Não useremos
price_duration|Preço parcial cobrado pela duração da viagem||Não useremos
distance|Distância percorrida|Unidade em Decímetros (1km = 10.000dm)|Não useremos
duration|Tempo total gasto na viagem||Não useremos
cost|Comissão do Motorista|75% MOTORISTA \| 15% UBER|
cost_distance|Custo da Distancia||Não useremos
cost_duration|Custo da Duração||Não useremos
source|Dispositivo do usuário que solicitou a corrida|iPhone, web, Android, iPad, Wap|
driver_score|Pontuação de 1 a 5 estrelas dada ao motorista||
rider_score|Pontuação de 1 a 5 estrelas dada ao passageiro||


STATUS DA VIAGEM
end_state|descrição|
---|---
drop off|Corrida Com Sucesso||
rider cancel|Passageiro Cancelou||
not found|Não Encontrado (motor/passag)||
driver cancel|Motorista Cancelou||
failure|Falha||
no show|Passageiro Não Apareceu||


TIPO DE VIAGEM
start_type|descrição|
---|---
asap|Normal (assim que possível)||
reserved|Reservada/Agendada||
delayed|Imediata Com Prioridade||


TIPO DE VEÍCULO (base: média de preços das corridas)
icon|descrição|
---|---
executive|Executivo (mais baratas)||
group|Grupo (Mais caras)||
easy|Simples (Intermediária)||



Irei fazer uma copia do DF como DF2, somente com as variáveis que utilizarei. Gosto de manter o arquivo original para consultar, se necessário.

In [41]:
# o comando .drop exclui as colunas do df e o parametro axis=1 indica que desejo excluir as colunas
df2 = df.drop(['journey_id', 'user_id', 'driver_id', 'taxi_id', 'arrived_at', 'price_distance', 'price_duration', 'distance', 'duration', 'cost_distance', 'cost_duration'], axis = 1)
display(df2.head(3))
display(df2.info())

,icon,start_type,start_at,start_lat,start_lon,end_at,end_lat,end_lon,end_state,driver_start_lat,driver_start_lon,currency,price,cost,source,driver_score,rider_score
0,executive,asap,16/11/2010 16:44,"-12,13983536","-77,02355957",16/11/2010 17:29,"-12,05537033","-77,0413208",drop off,"-12,10825481","-77,0272739",PEN,3821.0,0.0,iPhone,5.0,5.0
1,executive,asap,01/06/2010 0:34,"-12,13874817","-76,99536133",01/06/2010 0:37,"-12,12465668","-77,02886963",not found,NaN,NaN,PEN,0.0,0.0,iPhone,NaN,NaN
2,executive,asap,31/05/2010 5:01,"-12,12453079","-77,02780151",31/05/2010 5:04,"-12,13883209","-76,99530029",not found,NaN,NaN,PEN,0.0,0.0,iPhone,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23111 entries, 0 to 23110
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   icon              23111 non-null  object 
 1   start_type        23111 non-null  object 
 2   start_at          23111 non-null  object 
 3   start_lat         23111 non-null  object 
 4   start_lon         23111 non-null  object 
 5   end_at            22835 non-null  object 
 6   end_lat           23111 non-null  object 
 7   end_lon           23111 non-null  object 
 8   end_state         23099 non-null  object 
 9   driver_start_lat  19621 non-null  object 
 10  driver_start_lon  19621 non-null  object 
 11  currency          23111 non-null  object 
 12  price             22713 non-null  float64
 13  cost              21760 non-null  float64
 14  source            22988 non-null  object 
 15  driver_score      7650 non-null   float64
 16  rider_score       15390 non-null  float6

None